# Demo 5A #
The purpose of this demonstration is reiterate topics covered in Lectures 5.1 and 5.2. After this demonstration, you should feel comfortable:
- implementing an LDA model with `sklearn`
- assessing the presence of topics in elements of the corpus
- evaluating individual topics
- computing model and topic evaluation metrics
- tuning LDA models

## The Data ##
In the spirit of the example used in the lectures, we're going to use some complaints data to illustrate these models. I'm using data from the Consumer Financial Protection Bureau, which you can find __[here](https://www.consumerfinance.gov/data-research/consumer-complaints/#download-the-data)__. I've provided the compressed, JSON formatted file. If you inspect the text file, you'll see it is a list of JSON records.

In [2]:
import pandas as pd, numpy as np
from zipfile import ZipFile
import json

In [5]:
with ZipFile('complaints.json.zip','r') as z:
    complaints = json.loads(z.read('complaints.json'))

Let's get a sense as to what this data looks like:

In [8]:
print(f"There are {len(complaints)} complaints in this data.")
print("Sample Complaints:\n")
print(complaints[0])
print(complaints[1000])

There are 6169994 complaints in this data.
Sample Complaints:

{'date_received': '2024-09-15', 'product': 'Credit reporting or other personal consumer reports', 'sub_product': 'Credit reporting', 'issue': 'Improper use of your report', 'sub_issue': 'Reporting company used your report improperly', 'complaint_what_happened': '', 'company_public_response': '', 'company': 'Paramount Recovery Systems, L.P.', 'state': 'FL', 'zip_code': '33578', 'tags': '', 'consumer_consent_provided': '', 'submitted_via': 'Web', 'date_sent_to_company': '2024-09-15', 'company_response': 'Closed with explanation', 'timely': 'Yes', 'consumer_disputed': 'N/A', 'complaint_id': '10131769'}
{'date_received': '2024-07-22', 'product': 'Credit reporting or other personal consumer reports', 'sub_product': 'Credit reporting', 'issue': 'Incorrect information on your report', 'sub_issue': 'Information belongs to someone else', 'complaint_what_happened': '', 'company_public_response': '', 'company': 'Experian Information S

The actual text of the complaint is in "complaint_what_happened". We'll create a dataframe and then retain only those complaints with that full text:

In [11]:
complaintsDF = pd.DataFrame(complaints)
complaintsDF = complaintsDF.loc[complaintsDF['complaint_what_happened']!=""]
len(complaintsDF)

2115520

This culls about 2/3 of the data, but to keep this manageable let's pull a random sample of 10,000 complaints:

In [13]:
complaintsDF = complaintsDF.sample(10000,random_state=123)

In [14]:
complaintsDF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 1144439 to 67420
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   date_received              10000 non-null  object
 1   product                    10000 non-null  object
 2   sub_product                10000 non-null  object
 3   issue                      10000 non-null  object
 4   sub_issue                  10000 non-null  object
 5   complaint_what_happened    10000 non-null  object
 6   company_public_response    10000 non-null  object
 7   company                    10000 non-null  object
 8   state                      10000 non-null  object
 9   zip_code                   10000 non-null  object
 10  tags                       10000 non-null  object
 11  consumer_consent_provided  10000 non-null  object
 12  submitted_via              10000 non-null  object
 13  date_sent_to_company       10000 non-null  object
 14  compa

Recall the steps for running a topic model:
1. Preprocessing
2. Feature Extraction
3. Model Selection
4. Model Tuning & Evaluation
5. Communicate Results

We're going to use `sklearn` which allows us to combine some preprocessing steps within our feature extraction procedure.

## Preprocessing & Feature Extraction ##
We're going to generate a document term matrix with the following criteria:
1. Include only letters and require at least 3 characters per token
2. Remove stop words and dates (add XXXX and XX/XX/XXXX to stop words list)
3. Consider bigrams or single words
4. Retain only the 1,000 most common features

As we've seen in previous demos, `sklearn` makes it very easy to incorporate these steps.

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

# load stop words
stops = set(stopwords.words('english'))
stops.add("xxxx")
stops.add("xx/xx/xxxx")

# set up vectorizer
vec = CountVectorizer(token_pattern = r'\b[a-zA-Z]{3,}\b',
                     stop_words = list(stops), # newer versions require stops to be list
                     ngram_range = (1,2),
                     max_features = 1000)

Now we will generate our document-term matrix:

In [23]:
dtm = vec.fit_transform(complaintsDF['complaint_what_happened'])

In [25]:
dtm

<10000x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 440968 stored elements in Compressed Sparse Row format>

Most common words:

In [30]:
for idx in reversed(dtm.todense().sum(axis=0).argsort()[:,-20:].tolist()[0]): # create dense matrix, compute word counts, use argsort, grab last 20 indices, convert to list (will be nested list, so take first element), pull feature names; reversed flips order
    print(vec.get_feature_names_out()[idx])

credit
account
information
report
consumer
reporting
credit report
payment
accounts
section
debt
would
also
bank
received
company
loan
balance
please
card


## LDA with `sklearn`

As I've mentioned several times, scikit-learn has a very simple, intuitive API that makes it very easy to fit various models. The LDA model is available in `sklearn.decomposition`, and you can find the documentation for the model __[here](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html)__. Let's load the model and then we'll talk about the parameters we're going to set:

In [34]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

If you review the documentation, you'll find many different parameters we can set. We're going to set a few of them. In practice, you would likely want to "tune" many of these (try lots of different values and evaluate model fit). We're going to focus on a few:

1. `n_components`: The number of topics (most important!)
2. `doc_topic_prior`: This is "alpha" from the slides. The default value is 1/n_components (the number of topics). If you search around you'll find that, more typically, something like "50/T" is appropriate, where "T" is the number of topics. We will split the difference and go with 25/T
3. `topic_word_prior`: This is "beta" from the slides. The default value is the same as for alpha, but if you look at other references many say to use something like 0.1 or 200/W, where W is the number of words (so 0.2 in our setting). We'll go with 0.15 to start.
4. `n_jobs`: This allows you to parallelize. "-1" will use all cores.

For simplicity, we'll leave the rest of the parameters the same. Go ahead and run this cell and come back when it's complete. It should take a couple of minutes.

In [37]:
topics = 100

lda = LDA(n_components = topics,
         doc_topic_prior = 25/topics,
         topic_word_prior = 0.15,
         n_jobs = -1,
         random_state=123)

doc_topics = lda.fit_transform(dtm)

In [39]:
doc_topics

array([[0.00185703, 0.00410383, 0.00186805, ..., 0.00196642, 0.15632024,
        0.03958817],
       [0.00463227, 0.00470617, 0.02292442, ..., 0.00477823, 0.00469649,
        0.00474707],
       [0.00741588, 0.00697513, 0.00696231, ..., 0.00699366, 0.00694445,
        0.00699268],
       ...,
       [0.00265879, 0.00246359, 0.00252948, ..., 0.00265572, 0.00250008,
        0.00261666],
       [0.00574988, 0.0055684 , 0.00560237, ..., 0.02086514, 0.00560528,
        0.14916491],
       [0.00716393, 0.00698338, 0.00680025, ..., 0.00678511, 0.00678024,
        0.04308231]])


#### Exploring the Topic-Document Matrix 
Let's take a quick peek at our output; it should be 10,000 documents (our sample) by 100 columns (number of topics):

In [42]:
doc_topics.shape

(10000, 100)

Note that the topic-relevance scores are probabalistic and will sum to 1:

In [45]:
doc_topics.sum(axis=1)

array([1., 1., 1., ..., 1., 1., 1.])

Recall that it is up to us to define a threshold for when a topic is actually relevant to a document. Let's set the threshold at 0.10 and examine (1) how many topics appear in each document, and (2) which topic identifiers are most common

In [48]:
threshold = 0.10
dt2 = (doc_topics>threshold).astype(int)
dt2

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]])

In [50]:
# (1) Topics per document:
dt2.sum(axis=1).mean()

1.5288

In [52]:
# (2) topics that are most common:
dt2.sum(axis=0).argsort()[-10:]

array([74, 62, 85, 41, 20, 91, 76, 44, 26, 24])

How do we know what constitutes topic 3, for instance?

#### Exploring Topics (with Topic-Word Matrix)

One drawback of `sklearn`'s implementation of LDA, particularly as compared to `gensim`, is that it does not provide an easy way of accessing words relevant to each topic. However, we can use some simple numpy methods to access information in the topic-word matrix. We'll define a function to access this information in a moment, but let's start by exploring the underlying data we'll use:

In [57]:
vocab = np.array(vec.get_feature_names_out()) # the vocabulary of our DTM; storing as numpy array to make it easier to access multiple elements with a list (i.e., I can say vocab[[1,2,3,...]])
top_word = lda.components_
top_word

array([[ 0.18042992,  0.15050105,  0.15018962, ...,  0.15052758,
         0.15001005,  0.15045665],
       [ 0.17237487,  0.1503968 ,  0.15195612, ...,  0.55118859,
         0.15000161,  0.1507837 ],
       [ 1.36547275,  0.32566894,  0.15141976, ...,  0.15056611,
         0.15000118,  4.32438008],
       ...,
       [ 0.15039791,  6.87093012,  0.15098516, ...,  0.73921355,
         0.15002264, 33.47698058],
       [ 0.15081336,  0.15048058,  0.15041882, ..., 23.45218189,
         0.15000208,  3.62057035],
       [ 0.15224601,  0.15046106,  8.47410813, ...,  0.15065494,
         0.15000417,  0.15060303]])

This matrix has dimensions 100 (topics) by 1000 (words). Higher values indicate a greater probability of the word being relevant for a given topic. So, to identify words that are most relevant for a given topic, we need to identify the highest values and use the column indices to access the vocabulary. Let's use topic 3 as an example:

In [60]:
topic = 3
# the word vector for that topic:
top_word[3,:]

array([1.67695435e+00, 1.50530984e-01, 1.50205662e-01, 1.50117855e-01,
       2.76672271e-01, 1.50337960e-01, 1.66602764e-01, 1.78350628e+03,
       6.06471907e+01, 1.50060296e-01, 1.50128787e-01, 1.50464774e-01,
       1.24016543e+01, 1.12273048e+03, 1.50169151e-01, 1.50283446e-01,
       1.50037922e-01, 1.50422996e-01, 3.26171423e+00, 1.50378173e-01,
       1.50159955e-01, 1.50337012e-01, 1.50065352e-01, 1.50412377e-01,
       1.50687703e-01, 1.50378494e-01, 4.00736406e-01, 1.50095817e-01,
       1.50198760e-01, 1.50413416e-01, 1.50752270e-01, 1.50409709e-01,
       1.50430854e-01, 1.50477363e-01, 1.50343440e-01, 1.50577894e-01,
       1.50519635e-01, 1.50355922e-01, 1.50789015e-01, 1.50277251e-01,
       3.00287711e-01, 1.50124492e-01, 1.52499488e-01, 1.50243491e-01,
       1.50230199e-01, 1.50205797e-01, 7.16233378e-01, 1.50201448e-01,
       1.50290407e-01, 1.50184088e-01, 1.50267691e-01, 1.50330138e-01,
       1.50382150e-01, 1.50551862e-01, 1.50348236e-01, 1.50165187e-01,
      

In [62]:
# 10 most relevant words (from most to least relevant with [::-1])
top_words = top_word[3,:].argsort()[-20:][::-1].tolist()

In [64]:
top_words

[597,
 7,
 13,
 598,
 573,
 8,
 382,
 456,
 244,
 580,
 574,
 749,
 12,
 751,
 744,
 993,
 62,
 441,
 215,
 485]

Finally, we can use this top_words item to access the words in the vocabulary corresponding to this topic.

In [67]:
vocab[top_words]

array(['number', 'account', 'account number', 'number account', 'name',
       'account account', 'following', 'information', 'credit', 'needs',
       'name account', 'reported', 'account name', 'reporting', 'report',
       'xxxxxxxx', 'also', 'inaccurate', 'consumer', 'item'], dtype=object)

This sounds like a coherent topic relating to bank accounts, and more specifically transferring funds or closing accounts.

Let's now transfer this code to a function we can use to access the words for any topic:

In [70]:
def get_topic_words(topic,top_word,vocab,topn=10):
    top_words = top_word[topic,:].argsort()[-topn:][::-1].tolist()
    return vocab[top_words]

# Test:
get_topic_words(3,top_word,vocab,topn=5)

array(['number', 'account', 'account number', 'number account', 'name'],
      dtype=object)

Now let's iterate over the topics and print out the 5 most common words for each:

In [73]:
for top in range(topics):
    words = get_topic_words(top,top_word,vocab,topn=5)
    print(f"Topic {top}: {'|'.join(words)}")

Topic 0: accurate|information|update|credit|investigate
Topic 1: attached|see|investigation|want|inc
Topic 2: business|days|business days|within|complaint
Topic 3: number|account|account number|number account|name
Topic 4: data|caused|breach|affected|continues
Topic 5: account|access|branch|opened|information
Topic 6: money|back|said|get|hold
Topic 7: security|social|social security|number|name
Topic 8: bureaus|credit bureaus|multiple|credit|multiple times
Topic 9: experian|local|behalf|also|credit
Topic 10: inquiry|authorized|account|remove credit|credit report
Topic 11: information|credit|creditors|properly|inaccurate
Topic 12: bank|like|would|would like|bank account
Topic 13: usc|information|inaccurate|consumer|according
Topic 14: legal|items|action|legal action|take
Topic 15: fraudulent|credit|report|possible|accounts
Topic 16: account|incorrect|information|account account|incorrect information
Topic 17: said|asked|would|stating|letter
Topic 18: court|debt|collections|show|collect


This is subjective, but overall this seems to have done a pretty good job! Now let's explore how we might evaluate model and topic quality.

### Evaluating Model & Topic Quality
In the lectures, we talked about several different approaches to evaluating model quality. At the model level, we talked about perplexity. We compute perplexity as follows:

In [ ]:
perplexity = lda.perplexity(dtm)
perplexity

This really isn't interpretable on its own--it's meant to be compared to other models (or used in the elbow method). We'll come back to this later (there's actually a bug in `sklearn`'s perplexity).

As for evaluating the quality of the topics, there are numerous metrics out there but none are built into `sklearn` directory. The other main LDA implementation in Python, available through `gensim`, does have many different metrics available. While we cannot use `gensim` methods and functions on `sklearn` objects, there is a package available that provides a link between the two, `tmtoolkit`. We'll compute one metric, called "u_mass", which is one of the indices that evaluates the likelihood words in a topic co-occur.

In [ ]:
from tmtoolkit.topicmod.evaluate import metric_coherence_gensim

In [ ]:
umass = metric_coherence_gensim(measure = "u_mass", # essentially a measure of how likely a word pair in a topic is in a corpus, higher is better (but will be negative)
                        top_n = 5,
                        topic_word_distrib = top_word,
                        dtm = dtm.todense(),
                        vocab = vocab,
                        texts = None) # note that metrics other than "u_mass" require texts to be passed as tokenized text in a list of lists (because they use sliding windows)

In [ ]:
print(umass[:10]) # by topic
print(np.mean(umass))

These metrics are relatively easy to calculate, but they they can sometimes suggest a different level of quality than what a human would interpret. Word intrusion tasks can be very helpful for evaluating topic quality.

Let's set up a word intrusion task by doing the following:
(1) Collect the top 5 words associated with each topic
(2) Add a random word from the rest of the vocabulary

In practice, we'd then randomize the order of the 5 words, but we won't do that so we can evaluate the intruder vs. other words.

In [ ]:
import random
random.sample(list(vocab),1)[0]

In [ ]:
import random
for top in range(topics):
    words = get_topic_words(top,top_word,vocab,topn=5)
    random_word = random.sample(list([w for w in vocab if w not in words]),1)[0]
    print(f"Topic {top} words: {'|'.join(words)}\t\tIntruder: {random_word}")

Overall, it seems like the intruders are pretty clearly different than the other topic words with few exceptions.

### Tuning the model ###
The last thing we will cover (briefly) is how you might go about tuning this LDA model. In practice, you would likely want to try a variety of different parameters, and `sklearn` offers something called a `GridSearchCV` (or `RandomizedSearchCV`) that facilitates trying various combinations of parameters. For brevity, though, we're just going to vary the number of topics and evaluate the two metrics we just examined. Specifically, we'll consider topics between 50 and 150, counting by 10s. We'll collect the metrics and evaluate the results at the end:

In [ ]:
records = []
for top in range(50,151,10):
    print(f"Fitting {top} topics")
    record = {'topics':top}
    lda = LDA(n_components = top,
             doc_topic_prior = 25/top,
             topic_word_prior = 0.15,
             n_jobs = -1,
             random_state=123)

    lda.fit(dtm) # note I'm just "fitting" here, not "fit_transform"
    record['perplexity'] = lda.perplexity(dtm)
    umass = metric_coherence_gensim(measure = "u_mass",
                        top_n = 5,
                        topic_word_distrib = lda.components_,
                        dtm = dtm.todense(),
                        vocab = vocab,
                        texts = None) 
    record['mean_umass'] = np.mean(umass)
    records.append(record)

diagnostics = pd.DataFrame(records)

In [ ]:
diagnostics

In [ ]:
import matplotlib.pyplot as plt
fig,ax = plt.subplots()
ax2 = ax.twinx()
ax.plot(diagnostics['topics'],diagnostics['perplexity'])
ax2.plot(diagnostics['topics'],diagnostics['mean_umass'],color='red',linestyle='--')

The plot seems to suggest that there is a lot of variation in both of these scores, but note the scale--this is very zoomed in! It actally seems like things are relatively flat.

In addition, perplexity in scikit-learn's implementation has a known issue--it should not be increasing with the number of topics. In my experience, it seems to be inverted--you can still use the elbow method, but the plot is flipped. However, most references suggest using other means to pick the best model, like the coherence measure we computed, along with your own intuition.

### Final Task 

We focused completely on LDA in this demo, but we also covered NMF. On your own, I'd like you to try to fit an NMF model. You can use the same settings we used in our first LDA model (100 topics, simple word counts). You should be able to use the same function we wrote above to examine the topics produced by NMF. The documentation for NMF can be found __[here](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html)__. 

**HOMEWORK** (for disucssion): What are the 5 words associated with the most prevalent topic under this method using the 10% threshold we used previously (**NOTE**: Unlike LDA, NMF does not produce topic probabilities by document)? To transform to something that looks like LDA probabilities, or numbers summing to 1 by row, run `(res / res.sum(axis=1)[:,None])` on your document-topic matrix (assumed to be `res` in that code). Discuss your answer to this question, and the perceived quality of topics on the discussion board (no need to run coherence scores, etc.). 

I'll help you get started:

In [ ]:
from sklearn.decomposition import NMF
nmf = NMF(n_components=100, random_state=123) # while NMF can be deterministic, sklearn has several different intialization options, some of which have a random element.